<center><h1>Google AI Assistant for Data Science teaching</h1></center>

<center><img src="https://res.infoq.com/news/2024/02/google-gemma-open-model/en/headerimage/generatedHeaderImage-1708977571481.jpg" width="400"></center>

# Introduction

This Notebook will build an AI Assistant for Data Science teaching.

## The method

We will fine-tune a LLM with Data Science questions and answers. Then we will create a custom class that will query this model.

## The model

As model, we will use Gemma model, fine-tuned with our Data Science Q&A data, using LoRA.

## The data

As data for fine-tuning Gemma, we will use [Data Science Q&A Treasury](https://www.kaggle.com/datasets/memocan/data-science-interview-q-and-a-treasury) data.

## Previous work

This work is largely based on previous work. Here I list the sources:

1. Gemma Model Card, Kaggle Models, https://www.kaggle.com/models/google/gemma
2. Kaggle QA with Gemma - KerasNLP Starter, Kaggle Code, https://www.kaggle.com/code/awsaf49/kaggle-qa-with-gemma-kerasnlp-starter (Version 11)
3. Fine-tune Gemma models in Keras using LoRA, Kaggle Code, https://www.kaggle.com/code/nilaychauhan/fine-tune-gemma-models-in-keras-using-lora (Version 1)
4. Edward J. Hu, Yelong Shen, Phillip Wallis, Zeyuan Allen-Zhu, Yuanzhi Li, Shean Wang, Lu Wang, Weizhu Chen, LoRA: Low-Rank Adaptation of Large Language Models, ArXiv, https://arxiv.org/pdf/2106.09685.pdf
5. Abheesht Sharma, Matthew Watson, Parameter-efficient fine-tuning of GPT-2 with LoRA, https://keras.io/examples/nlp/parameter_efficient_finetuning_of_gpt2_with_lora/
6. Keras 3 API documentation / KerasNLP / Models / Gemma, https://keras.io/api/keras_nlp/models/gemma/


# Introduction about Gemma


Gemma is a collection of lightweight source generative AI models designed to be used mostly by developers and researchers. Created by Google DeepMind research lab that also developed Gemini, Gemma is available in several versions, with 2B and 7B parameters, as following:

| Model                  | Parameters      | Tuned versions    | Description                                    | Recomemnded target platforms       |
|------------------------|-----------------|-------------------|------------------------------------------------|------------------------------------|
| `gemma_2b_en`          | 2.51B           | Pretrained        | 18-layer Gemma model (Gemma with 2B parameters)|Mobile devices and laptops          |
| `gemma_instruct_2b_en` | 2.51B           | Instruction tuned | 18-layer Gemma model (Gemma with 2B parameters)| Mobile devices and laptops         | 
| `gemma_7b_en`          | 8.54B           | Pretrained        | 28-layer Gemma model (Gemma with 7B parameters)| Desktop computers and small servers|
| `gemma_instruct_7b_en` | 8.54B           | Instruction tuned | 28-layer Gemma model (Gemma with 7B parameters)| Desktop computers and small servers|


For this notebook, we will fine-tune `gemma_2b_en` model.

# LoRA introduction

LoRA stands for Low-Rank Adaptation. It is a method used to fine-tune large language models (LLMs) by freezing the weights of the LLM and injecting trainable rank-decomposition matrices. The number of trainable parameters during fine-tunning will decrease therefore considerably. According to LoRA paper, this number decreases 10,000 times, and the computational resources size decreases 3 times. 

# Installations and configurations

In [1]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.1.1 which is incompatible.


In [2]:
import os
os.environ["KERAS_BACKEND"] = "jax" # you can also use tensorflow or torch
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00" # avoid memory fragmentation on JAX backend.

import keras
import keras_nlp

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas() # progress bar for pandas

import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown

2024-03-31 21:42:29.540505: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-31 21:42:29.540623: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-31 21:42:29.667879: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
class Config:
    seed = 42
    dataset_path = "/kaggle/input/data-science-interview-q-and-a-treasury/dataset.csv"
    preset = "gemma_2b_en" # name of pretrained Gemma
    sequence_length = 512 # max size of input sequence for training
    batch_size = 1 # size of the input batch in training, x 2 as two GPUs
    epochs = 15 # number of epochs to train

In [4]:
keras.utils.set_random_seed(Config.seed)

In [5]:
def colorize_text(text):
    for word, color in zip(["Question", "Answer"], ["blue", "red"]):
        text = text.replace(f"\n\n{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

# Load the data

In [6]:
df = pd.read_csv(f"{Config.dataset_path}")
df.head()

,question,answer
0,What is supervised machine learning? 👶,Supervised learning is a type of machine learn...
1,What is regression? Which models can you use t...,Regression is a part of supervised ML. Regress...
2,What is linear regression? When do we use it? 👶,Linear regression is a model that assumes a li...
3,What are the main assumptions of linear regres...,There are several assumptions of linear regres...
4,What’s the normal distribution? Why do we care...,The normal distribution is a continuous probab...


# Fine-tune Gemma

In [7]:
template = "\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"
df["prompt"] = df.progress_apply(lambda row: template.format(question=row.question,
                                                             answer=row.answer), axis=1)
data = df.prompt.tolist()

  0%|          | 0/166 [00:00<?, ?it/s]

## Initialize the code for Gemma Causal LM

In [8]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")
gemma_lm.summary()

Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

## Gemma preprocessor

In [9]:
x, y, sample_weight = gemma_lm.preprocessor(data[0:1])

## Enable LoRA for the model

In [10]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,507,536,384 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,507,536,384 (9.34 GB)

 Trainable params: 1,363,968 (5.20 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

## Run the fine-tuning sequence

In [11]:
# Limit the input sequence length to 512 (to control memory usage).
gemma_lm.preprocessor.sequence_length = Config.sequence_length 

# Compile the model with loss, optimizer, and metric
gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=8e-5),
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train model
gemma_lm.fit(data, epochs=Config.epochs, batch_size=Config.batch_size)

Epoch 1/15
166/166 ━━━━━━━━━━━━━━━━━━━━ 143s 736ms/step - loss: 0.5156 - sparse_categorical_accuracy: 0.5393
Epoch 2/15
166/166 ━━━━━━━━━━━━━━━━━━━━ 121s 730ms/step - loss: 0.4363 - sparse_categorical_accuracy: 0.5805
Epoch 3/15
166/166 ━━━━━━━━━━━━━━━━━━━━ 121s 730ms/step - loss: 0.4144 - sparse_categorical_accuracy: 0.5900
Epoch 4/15
166/166 ━━━━━━━━━━━━━━━━━━━━ 121s 730ms/step - loss: 0.4016 - sparse_categorical_accuracy: 0.6002
Epoch 5/15
166/166 ━━━━━━━━━━━━━━━━━━━━ 121s 730ms/step - loss: 0.3925 - sparse_categorical_accuracy: 0.6070
Epoch 6/15
166/166 ━━━━━━━━━━━━━━━━━━━━ 143s 860ms/step - loss: 0.3820 - sparse_categorical_accuracy: 0.6161
Epoch 7/15
166/166 ━━━━━━━━━━━━━━━━━━━━ 121s 730ms/step - loss: 0.3689 - sparse_categorical_accuracy: 0.6251
Epoch 8/15
166/166 ━━━━━━━━━━━━━━━━━━━━ 121s 730ms/step - loss: 0.3534 - sparse_categorical_accuracy: 0.6368
Epoch 9/15
166/166 ━━━━━━━━━━━━━━━━━━━━ 143s 860ms/step - loss: 0.3356 - sparse_categorical_accuracy: 0.6475
Epoch 10/15
166/166

# Test the fine-tuned model

## Define the specialized class

In [12]:
class GemmaQA:
    def __init__(self, max_length=512):
        self.max_length = max_length
        self.prompt = template
        self.gemma_lm = gemma_lm
        
    def query(self, question):
        response = self.gemma_lm.generate(
            self.prompt.format(
                question=question,
                answer=""), 
            max_length=self.max_length)
        display(Markdown(colorize_text(response)))
        

In [13]:
gemma_qa = GemmaQA()

## Test 1

In [14]:
row = df.iloc[5]
gemma_qa.query(row.question)



**<font color='blue'>Question:</font>**
How do we check if a variable follows the normal distribution? ‍⭐️

**<font color='red'>Answer:</font>**
* Skewness is a measure of the distribution’s skew. Skewed distributions have a skewness of 1 or more. If the skewness is negative, the curve is to the left of the mean and the distribution is said to be “left skewed.” If the skewness is positive, the curve is to the right of the mean and the distribution is said to be “right skewed.”
* Kurtosis is a measure of the degree of heaviness (or thinness) of the peak of the distribution. Kurtosis is a measure of how heavy-tailed the distribution is. Distributions with heavy peaks have higher values in kurtosis, and therefore, the peak is said to be “peaked”. Distributions that have a thin peak have lower values in kurtosis, and therefore, the peak is said to be “flat.”

## Test 2

In [15]:
row = df.iloc[10]
gemma_qa.query(row.question)



**<font color='blue'>Question:</font>**
What is SGD  —  stochastic gradient descent? What’s the difference with the usual gradient descent? ‍⭐️

**<font color='red'>Answer:</font>**
Answer here

# Test 3

In [16]:
row = df.iloc[15]
gemma_qa.query(row.question)



**<font color='blue'>Question:</font>**
How to validate your models? 👶

**<font color='red'>Answer:</font>**
Validation refers to a process where we test the generalizability of our model. We want to check whether it can make predictions for new, unseen data.
For a machine learning model to perform well, we need to make sure that the model doesn't overfit the data.
We can use the following evaluation metrics while validating the model:
1. Cross-validation: It is used to test the generalizability of the model.
2. Test-set validation: It is used when we have a huge amount of data and don't want to test the model's generalizability.
3. Grid search: It is used in case of overfitting when you have a large number of hyper-parameters to tune.
4. Out-of-sample error : It is used when we have a limited amount of data and don't want to test the model's generalisatio.

## Fresh question 1

In [17]:
question = "What is regularization?"
gemma_qa.query(question)



**<font color='blue'>Question:</font>**
What is regularization?

**<font color='red'>Answer:</font>**
Regularization is a technique that is used to prevent the overfitting of the model. It is used to prevent the model to learn too much from the training data and make the model general and robust. There are various regularization techniques like Lasso, Ridge, and LightGBM regularization but the most commonly known regularization technique is Shrinkage.

## Fresh question 2

In [18]:
question = "What is SVM?"
gemma_qa.query(question)



**<font color='blue'>Question:</font>**
What is SVM?

**<font color='red'>Answer:</font>**
SVM stands for support vector machines. It is a classification method that tries to find a separating hyperplane in a high dimensional space to accurately classify the data. It is commonly used in many real-world applications.

## Fresh question 3

In [19]:
question = "What is Dropout?"
gemma_qa.query(question)



**<font color='blue'>Question:</font>**
What is Dropout?

**<font color='red'>Answer:</font>**
Dropout is a technique that prevents a neural network from overfitting by temporarily removing some of the neurons during the training process. It helps the network to explore more of the parameter space, thus making it less prone to overfitting.

# Conclusions


We fine-tuned Gemma with a set of Data Science interview question and answers.   
Then we tested the model with questions from the dataset used for fine-tuning.  
At the end, we also tested with some questions that were not in the dataset.